In [1]:
!pip install mlflow

   ---------------------------------------- 0.0/20.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/20.2 MB 1.9 MB/s eta 0:00:11
   ---------------------------------------- 0.1/20.2 MB 1.1 MB/s eta 0:00:20
   ---------------------------------------- 0.2/20.2 MB 1.7 MB/s eta 0:00:12
    --------------------------------------- 0.4/20.2 MB 2.4 MB/s eta 0:00:09
   - -------------------------------------- 0.5/20.2 MB 2.4 MB/s eta 0:00:09
   - -------------------------------------- 1.0/20.2 MB 3.7 MB/s eta 0:00:06
   -- ------------------------------------- 1.3/20.2 MB 4.1 MB/s eta 0:00:05
   ---- ----------------------------------- 2.1/20.2 MB 6.2 MB/s eta 0:00:03
   ----- ---------------------------------- 2.9/20.2 MB 7.1 MB/s eta 0:00:03
   -------- ------------------------------- 4.5/20.2 MB 10.4 MB/s eta 0:00:02
   ----------- ---------------------------- 5.7/20.2 MB 11.3 MB/s eta 0:00:02
   --------------- ------------------------ 7.9/20.2 MB 14.8 MB/s eta 0:00:01
   


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ['GIT_PYTHON_REFRESH']='quiet'
os.environ['MLFLOW_TRACKING_URI']='file:///C:/Users/MLOPS'

In [3]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = iris_df.drop('target', axis=1)
y = iris_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

# Evaluate the model
predictions = rf.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [19]:
import mlflow
import mlflow.sklearn

# Log the model with MLflow
mlflow.set_tracking_uri(uri="file:///C:/Users/MLOPS")
EXPERIMENT_ID = mlflow.create_experiment("ex8",tags={"version":'v1','m1':'rf'})
exp=mlflow.set_experiment(experiment_name="ex8")
with mlflow.start_run():
    mlflow.sklearn.log_model(rf, "random-forest-model")
    mlflow.log_metric("accuracy", accuracy)

In [22]:
import requests
import json

#data = {"columns": iris.feature_names, "data": [[6.1, 2.8, 4.7, 1.2]]}
data = {'dataframe_records': X_test.values.tolist()}
headers = {"Content-Type": "application/json"}

response = requests.post("http://localhost:8001/invocations", data=json.dumps(data), headers=headers)
predictions = response.json()

print(predictions)

{'predictions': [1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]}


In [23]:
data

{'dataframe_records': [[6.1, 2.8, 4.7, 1.2],
  [5.7, 3.8, 1.7, 0.3],
  [7.7, 2.6, 6.9, 2.3],
  [6.0, 2.9, 4.5, 1.5],
  [6.8, 2.8, 4.8, 1.4],
  [5.4, 3.4, 1.5, 0.4],
  [5.6, 2.9, 3.6, 1.3],
  [6.9, 3.1, 5.1, 2.3],
  [6.2, 2.2, 4.5, 1.5],
  [5.8, 2.7, 3.9, 1.2],
  [6.5, 3.2, 5.1, 2.0],
  [4.8, 3.0, 1.4, 0.1],
  [5.5, 3.5, 1.3, 0.2],
  [4.9, 3.1, 1.5, 0.1],
  [5.1, 3.8, 1.5, 0.3],
  [6.3, 3.3, 4.7, 1.6],
  [6.5, 3.0, 5.8, 2.2],
  [5.6, 2.5, 3.9, 1.1],
  [5.7, 2.8, 4.5, 1.3],
  [6.4, 2.8, 5.6, 2.2],
  [4.7, 3.2, 1.6, 0.2],
  [6.1, 3.0, 4.9, 1.8],
  [5.0, 3.4, 1.6, 0.4],
  [6.4, 2.8, 5.6, 2.1],
  [7.9, 3.8, 6.4, 2.0],
  [6.7, 3.0, 5.2, 2.3],
  [6.7, 2.5, 5.8, 1.8],
  [6.8, 3.2, 5.9, 2.3],
  [4.8, 3.0, 1.4, 0.3],
  [4.8, 3.1, 1.6, 0.2]]}